In [1]:
import torch
from torch.distributions import uniform, normal
from Auxillaries import *
import math
import datetime
from ConfidenceIntervals import *

# Designing Model to be fitted

In [2]:
'''
Design of Model/Likelihood function to bet fitted
'''
def LogLikelihood(theta, x):

    '''
    This is the (log-)likelihood function that is supposed to be fitted. Note that this is the (log-)likelihood w.r.t.
    one data point X_i. Thus, in later calculations like gradient ascent updates, the average of this function will be considered.
    This can easily achieved by torch.mean(LogLikelihood(theta, X)) where X is the complete data set (X_1, ..., X_n).

    Info:
        - phi_torch(x, mu, sigma) gives the value of a density of a N(mu,sigma)-random variable at point x

    Arguments:
        - theta: parameter vector that needs to altered to fit the model
        - x: a datapoint or a data set: the output will have the same dimensions as this

    Output:
        - model log-likelihood of theta given x // element-wise on x

    '''

    g_theta = (1-theta[0, 0])*phi_torch(x, 0, 0.2) + theta[0, 0]*phi_torch(x, theta[0, 1], 0.2)
    log_g_theta = torch.log(g_theta)

    return log_g_theta

# Generating Data

In [3]:
'''
Create data of a gaussian mixture model
'''
# Specify parameters for mixture gaussian model
weights = [.5, .45, .05]
means = [[0.], [.75],[ 3.]]
covs = [[.2**2], [.2**2], [.2**2]]
n = 10**6

# Create sampels of given model
X = gaussian_mixture_model_sample(n, means, covs, weights, test=False) # we can alternatively use pytorchs MixtureFamily
X = torch.from_numpy(X)

# Contourplot of LogLikelihood-function

In [4]:
'''
Creating Contourlpot of likelihood function in dependence of parameters 
'''
# TODO: This is not well developed to be used for any setting: it needs limits for the parameters, works only for 2D, ... etc.
make_Contourplot = False
make_plots = True
ticks = 5000

if make_Contourplot:
    print('Creating Colourplot...')
    x_lin = np.linspace(0, 5, ticks)  # range for mu
    y_lin = np.linspace(0, 0.6, ticks) # range for rho

    #Colourplot = np.zeros((ticks, ticks))
    Colourplot = torch.zeros(ticks, ticks)

    for i in reversed(range(ticks)):
        for j in range(ticks):
            x = x_lin[j]
            y = y_lin[ticks - i - 1]
            #params = np.array([[y,x]])
            params = torch.tensor([[y,x]])
            Likelihoods = LogLikelihood(params, X) # dim 1xn
            L_value = torch.mean(Likelihoods, axis = 1) # dim 1xn
            Colourplot[i, j] = L_value

    np.save('Contourmatrix', Colourplot)
else:
    print('Loading Colourplot...')
    Colourplot = np.load('Contourmatrix.npy')

Loading Colourplot...


## Ground truth via epsilon netting

In [5]:
x_lin = np.linspace(0, 5, ticks)  # range for mu
y_lin = np.linspace(0, 0.6, ticks)
ind = np.unravel_index(np.argmax(Colourplot, axis=None), Colourplot.shape)
print(ind, y_lin[ticks-ind[0]-1], x_lin[ind[1]])

(1434, 1040) 0.427885577115423 1.0402080416083217


In [ ]:
# Approximating Ground truth with gradient Ascent

In [16]:
'''
Gradient Ascent
'''
#X = torch.from_numpy(X)
# Set parameters here
n_iterations = 5000 #max number of iterations # TODO: alternatively/ additionally a error-based threshold?
lr = 0.001 # learning rate / stepsize
n_runs = 10

# Initializing Loss as minus infinity to make sure first run achieves higher likelihood
max_likelihood = -1*np.inf
# trajectory_dict is a cache to save the gradient ascent trajectory of all gradient ascent runs
trajectory_dict = {}

theta_gt_list = []

# Running Gradient Ascent multiple (M=n_runs) times

for run in range(n_runs):
    
    # Create sampels of given model
    print('Sampling Data.../...\...')
    X = gaussian_mixture_model_sample(n, means, covs, weights, test=False) # we can alternatively use pytorchs MixtureFamily
    X = torch.from_numpy(X)
    print('Data sampled')
    
    # Create/ Initialize variable ' TODO: make initialization more flexible
    theta = torch.tensor([[0.43, 1.058]], requires_grad = True)

    # Run complete Gradient ascent
    theta, L, trajectory = gradient_ascent_torch2(func = LogLikelihood,
                                           param=theta,
                                           data=X,
                                           accuracy = 10**-16, 
                                           learningrate=lr,
                                           run_id=run,
                                           print_info=True)

    # Save optimization trajectory
    trajectory_dict.update({run : trajectory})
    theta_gt_list.append(theta.clone().data.numpy())

    # Updating Quantities if new max is found






Sampling Data.../...\...
Data sampled
Run: 1	| Iteration: 0 	| Log-Likelihood:-3.202937202009177 	|  theta: tensor([[0.4002, 1.0005]], requires_grad=True)  |  Time needed: 0:00:00.118971  
Run: 1	| Iteration: 100 	| Log-Likelihood:-3.190162779071696 	|  theta: tensor([[0.4132, 1.0325]], requires_grad=True)  |  Time needed: 0:00:14.666031  
Run: 1	| Iteration: 200 	| Log-Likelihood:-3.1875733949475653 	|  theta: tensor([[0.4202, 1.0464]], requires_grad=True)  |  Time needed: 0:00:14.247020  
Run: 1	| Iteration: 300 	| Log-Likelihood:-3.187008627886772 	|  theta: tensor([[0.4242, 1.0526]], requires_grad=True)  |  Time needed: 0:00:17.677741  
Run: 1	| Iteration: 400 	| Log-Likelihood:-3.186874831309641 	|  theta: tensor([[0.4266, 1.0552]], requires_grad=True)  |  Time needed: 0:00:13.073046  
Run: 1	| Iteration: 500 	| Log-Likelihood:-3.186838937082911 	|  theta: tensor([[0.4281, 1.0563]], requires_grad=True)  |  Time needed: 0:00:12.551092  
Run: 1	| Iteration: 600 	| Log-Likelihood:-3.

Run: 3	| Iteration: 1700 	| Log-Likelihood:-3.1648240129240417 	|  theta: tensor([[0.4308, 1.0547]], requires_grad=True)  |  Time needed: 0:00:15.721811  
Sampling Data.../...\...
Data sampled
Run: 4	| Iteration: 0 	| Log-Likelihood:-3.2251867751708403 	|  theta: tensor([[0.4002, 1.0005]], requires_grad=True)  |  Time needed: 0:00:00.181808  
Run: 4	| Iteration: 100 	| Log-Likelihood:-3.211263010292619 	|  theta: tensor([[0.4133, 1.0341]], requires_grad=True)  |  Time needed: 0:00:15.150564  
Run: 4	| Iteration: 200 	| Log-Likelihood:-3.2084523153994113 	|  theta: tensor([[0.4204, 1.0488]], requires_grad=True)  |  Time needed: 0:00:13.053035  
Run: 4	| Iteration: 300 	| Log-Likelihood:-3.2078440969151263 	|  theta: tensor([[0.4243, 1.0552]], requires_grad=True)  |  Time needed: 0:00:13.361296  
Run: 4	| Iteration: 400 	| Log-Likelihood:-3.2077026309571472 	|  theta: tensor([[0.4267, 1.0581]], requires_grad=True)  |  Time needed: 0:00:16.345809  
Run: 4	| Iteration: 500 	| Log-Likelihoo

Run: 6	| Iteration: 1500 	| Log-Likelihood:-3.174487079166337 	|  theta: tensor([[0.4310, 1.0559]], requires_grad=True)  |  Time needed: 0:00:16.202385  
Run: 6	| Iteration: 1600 	| Log-Likelihood:-3.174487072231044 	|  theta: tensor([[0.4311, 1.0558]], requires_grad=True)  |  Time needed: 0:00:16.603513  
Run: 6	| Iteration: 1700 	| Log-Likelihood:-3.174487039086576 	|  theta: tensor([[0.4311, 1.0558]], requires_grad=True)  |  Time needed: 0:00:16.518500  
Sampling Data.../...\...
Data sampled
Run: 7	| Iteration: 0 	| Log-Likelihood:-3.197429942371876 	|  theta: tensor([[0.4002, 1.0005]], requires_grad=True)  |  Time needed: 0:00:00.115161  
Run: 7	| Iteration: 100 	| Log-Likelihood:-3.1849061711115243 	|  theta: tensor([[0.4133, 1.0321]], requires_grad=True)  |  Time needed: 0:00:13.334673  
Run: 7	| Iteration: 200 	| Log-Likelihood:-3.182355215404251 	|  theta: tensor([[0.4205, 1.0458]], requires_grad=True)  |  Time needed: 0:00:16.584504  
Run: 7	| Iteration: 300 	| Log-Likelihood:

Run: 9	| Iteration: 1300 	| Log-Likelihood:-3.2088733115670633 	|  theta: tensor([[0.4302, 1.0607]], requires_grad=True)  |  Time needed: 0:00:10.638917  
Run: 9	| Iteration: 1400 	| Log-Likelihood:-3.2088733206081783 	|  theta: tensor([[0.4302, 1.0606]], requires_grad=True)  |  Time needed: 0:00:30.190431  
Run: 9	| Iteration: 1500 	| Log-Likelihood:-3.208873339638086 	|  theta: tensor([[0.4303, 1.0606]], requires_grad=True)  |  Time needed: 0:00:24.210433  
Run: 9	| Iteration: 1600 	| Log-Likelihood:-3.208873334521411 	|  theta: tensor([[0.4303, 1.0606]], requires_grad=True)  |  Time needed: 0:00:16.653636  
Run: 9	| Iteration: 1700 	| Log-Likelihood:-3.2088732723814335 	|  theta: tensor([[0.4303, 1.0606]], requires_grad=True)  |  Time needed: 0:00:18.468900  
Sampling Data.../...\...
Data sampled
Run: 10	| Iteration: 0 	| Log-Likelihood:-3.198502610739704 	|  theta: tensor([[0.4002, 1.0005]], requires_grad=True)  |  Time needed: 0:00:00.096612  
Run: 10	| Iteration: 100 	| Log-Likel

In [22]:
print(np.array(theta_gt_list).squeeze())

[[0.43105388 1.056834  ]
 [0.430388   1.0589968 ]
 [0.43079117 1.0547081 ]
 [0.43089142 1.0598941 ]
 [0.43154448 1.056336  ]
 [0.4310907  1.0558412 ]
 [0.43158382 1.0560336 ]
 [0.4309605  1.0574625 ]
 [0.43030012 1.0606308 ]
 [0.4313304  1.0566736 ]]


In [23]:
theta_1 = np.mean(theta_gt_list, axis = 0)
print(theta_1)

[[0.4309934 1.0573411]]


# Approximating local max with gradient ascent

In [4]:
'''
Gradient Ascent
'''
#X = torch.from_numpy(X)
# Set parameters here
n_iterations = 5000 #max number of iterations # TODO: alternatively/ additionally a error-based threshold?
lr = 0.001 # learning rate / stepsize
n_runs = 10

# Initializing Loss as minus infinity to make sure first run achieves higher likelihood
max_likelihood = -1*np.inf
# trajectory_dict is a cache to save the gradient ascent trajectory of all gradient ascent runs
trajectory_dict = {}

theta_loc_list = []

# Running Gradient Ascent multiple (M=n_runs) times

for run in range(n_runs):
    
    # Create sampels of given model
    print('Sampling Data.../...\...')
    X = gaussian_mixture_model_sample(n, means, covs, weights, test=False) # we can alternatively use pytorchs MixtureFamily
    X = torch.from_numpy(X)
    print('Data sampled')
    
    # Create/ Initialize variable ' TODO: make initialization more flexible
    theta = torch.tensor([[0.05, 3]], requires_grad = True)

    # Run complete Gradient ascent
    theta, L, trajectory = gradient_ascent_torch2(func = LogLikelihood,
                                           param=theta,
                                           data=X,
                                           accuracy = 10**-16, 
                                           learningrate=lr,
                                           run_id=run,
                                           print_info=True)

    # Save optimization trajectory
    trajectory_dict.update({run : trajectory})
    theta_loc_list.append(theta.clone().data.numpy())

    # Updating Quantities if new max is found






Sampling Data.../...\...
Data sampled
Run: 1	| Iteration: 0 	| Log-Likelihood:-3.1705492297770212 	|  theta: tensor([[0.0500, 3.0000]], requires_grad=True)  |  Time needed: 0:00:00.262389  
Run: 1	| Iteration: 100 	| Log-Likelihood:-3.1705490052486693 	|  theta: tensor([[0.0500, 2.9999]], requires_grad=True)  |  Time needed: 0:00:14.253233  
Run: 1	| Iteration: 200 	| Log-Likelihood:-3.170548890272284 	|  theta: tensor([[0.0499, 2.9998]], requires_grad=True)  |  Time needed: 0:00:12.945521  
Run: 1	| Iteration: 300 	| Log-Likelihood:-3.1705488178992436 	|  theta: tensor([[0.0499, 2.9997]], requires_grad=True)  |  Time needed: 0:00:13.759476  
Run: 1	| Iteration: 400 	| Log-Likelihood:-3.170548737393462 	|  theta: tensor([[0.0499, 2.9996]], requires_grad=True)  |  Time needed: 0:00:13.010718  
Run: 1	| Iteration: 500 	| Log-Likelihood:-3.170548681910487 	|  theta: tensor([[0.0499, 2.9995]], requires_grad=True)  |  Time needed: 0:00:16.769358  
Run: 1	| Iteration: 600 	| Log-Likelihood:-

Run: 5	| Iteration: 1400 	| Log-Likelihood:-3.177106652631698 	|  theta: tensor([[0.0497, 2.9993]], requires_grad=True)  |  Time needed: 0:00:09.074672  
Run: 5	| Iteration: 1500 	| Log-Likelihood:-3.177106648489257 	|  theta: tensor([[0.0497, 2.9993]], requires_grad=True)  |  Time needed: 0:00:09.130343  
Run: 5	| Iteration: 1600 	| Log-Likelihood:-3.177106645048649 	|  theta: tensor([[0.0497, 2.9993]], requires_grad=True)  |  Time needed: 0:00:09.127295  
Sampling Data.../...\...
Data sampled
Run: 6	| Iteration: 0 	| Log-Likelihood:-3.180844760167378 	|  theta: tensor([[0.0500, 3.0000]], requires_grad=True)  |  Time needed: 0:00:00.090040  
Run: 6	| Iteration: 100 	| Log-Likelihood:-3.1808440850709263 	|  theta: tensor([[0.0498, 2.9999]], requires_grad=True)  |  Time needed: 0:00:09.264271  
Run: 6	| Iteration: 200 	| Log-Likelihood:-3.1808440594870593 	|  theta: tensor([[0.0498, 2.9999]], requires_grad=True)  |  Time needed: 0:00:09.310339  
Run: 6	| Iteration: 300 	| Log-Likelihood

Run: 8	| Iteration: 1400 	| Log-Likelihood:-3.176982041719825 	|  theta: tensor([[0.0500, 2.9984]], requires_grad=True)  |  Time needed: 0:00:18.536151  
Run: 8	| Iteration: 1500 	| Log-Likelihood:-3.176982011654757 	|  theta: tensor([[0.0500, 2.9984]], requires_grad=True)  |  Time needed: 0:00:18.299808  
Run: 8	| Iteration: 1600 	| Log-Likelihood:-3.1769819870897513 	|  theta: tensor([[0.0500, 2.9983]], requires_grad=True)  |  Time needed: 0:00:17.564148  
Run: 8	| Iteration: 1700 	| Log-Likelihood:-3.1769819757948454 	|  theta: tensor([[0.0500, 2.9983]], requires_grad=True)  |  Time needed: 0:00:11.626445  
Run: 8	| Iteration: 1800 	| Log-Likelihood:-3.1769819614795205 	|  theta: tensor([[0.0500, 2.9983]], requires_grad=True)  |  Time needed: 0:00:13.593133  
Run: 8	| Iteration: 1900 	| Log-Likelihood:-3.1769819515943687 	|  theta: tensor([[0.0500, 2.9983]], requires_grad=True)  |  Time needed: 0:00:19.409664  
Run: 8	| Iteration: 2000 	| Log-Likelihood:-3.176981938688796 	|  theta:

In [5]:
print(np.array(theta_loc_list).squeeze())

[[0.04994409 2.9990084 ]
 [0.0497372  2.999747  ]
 [0.05008504 3.0001652 ]
 [0.04995714 3.0000162 ]
 [0.04967513 2.9992764 ]
 [0.04975339 2.9995391 ]
 [0.05007071 2.9984834 ]
 [0.04996041 2.9981716 ]
 [0.04976925 2.9985156 ]
 [0.05011119 2.9989712 ]]


In [7]:
theta_loc = np.mean(theta_loc_list, axis = 0)
print(theta_loc)

[[0.04990635 2.9991894 ]]


# Estimating q

In [34]:
'''
Gradient Ascent
'''
theta_gt = np.array([[0.4309934, 1.0573411]])
theta_loc = theta_loc
#X = torch.from_numpy(X)
# Set parameters here
n_iterations = 5000 #max number of iterations # TODO: alternatively/ additionally a error-based threshold?
lr = 0.01 # learning rate / stepsize
n_runs =1000

# Initializing Loss as minus infinity to make sure first run achieves higher likelihood
max_likelihood = -1*np.inf
# trajectory_dict is a cache to save the gradient ascent trajectory of all gradient ascent runs
trajectory_dict = {}

glob_max_counter = 0
loc_max_counter = 0
rest_counter = 0
# Running Gradient Ascent multiple (M=n_runs) times
for run in range(n_runs):

    # Create/ Initialize variable ' TODO: make initialization more flexible
    theta = torch.tensor([[uniform.Uniform(0., .6).sample(), uniform.Uniform(0., 5.).sample()]], requires_grad = True)

    # Run complete Gradient ascent
    theta, L, trajectory = gradient_ascent_torch(func = LogLikelihood,
                                           param=theta,
                                           data=X,
                                           max_iterations=1000,
                                           learningrate=lr,
                                           run_id=run,
                                           print_info=True)

    if np.linalg.norm(theta_gt-theta.clone().data.numpy())<0.1:
        glob_max_counter+=1
    elif np.linalg.norm(theta_loc-theta.clone().data.numpy())<0.1:
        loc_max_counter+=1
    else:
        rest_counter +=1
        



Run: 1	| Iteration: 0 	| Log-Likelihood:-3.442436769318044 	|  theta: tensor([[0.2288, 0.9123]], requires_grad=True)  |  Time needed: 0:00:00.123140  
Run: 1	| Iteration: 100 	| Log-Likelihood:-3.1904917286184666 	|  theta: tensor([[0.4291, 1.0580]], requires_grad=True)  |  Time needed: 0:00:09.876190  
Run: 1	| Iteration: 200 	| Log-Likelihood:-3.190462758827881 	|  theta: tensor([[0.4328, 1.0570]], requires_grad=True)  |  Time needed: 0:00:09.987045  
Run: 1	| Iteration: 300 	| Log-Likelihood:-3.1904627327526622 	|  theta: tensor([[0.4329, 1.0570]], requires_grad=True)  |  Time needed: 0:00:09.691370  
Run: 1	| Iteration: 400 	| Log-Likelihood:-3.190462702927815 	|  theta: tensor([[0.4329, 1.0570]], requires_grad=True)  |  Time needed: 0:00:10.061423  
Run: 1	| Iteration: 500 	| Log-Likelihood:-3.190462702927815 	|  theta: tensor([[0.4329, 1.0570]], requires_grad=True)  |  Time needed: 0:00:09.865562  
Run: 1	| Iteration: 600 	| Log-Likelihood:-3.190462702927815 	|  theta: tensor([[0

Run: 6	| Iteration: 400 	| Log-Likelihood:-3.187249991221496 	|  theta: tensor([[0.0501, 2.9945]], requires_grad=True)  |  Time needed: 0:00:09.476408  
Run: 6	| Iteration: 500 	| Log-Likelihood:-3.1872387590561537 	|  theta: tensor([[0.0501, 2.9976]], requires_grad=True)  |  Time needed: 0:00:09.356054  
Run: 6	| Iteration: 600 	| Log-Likelihood:-3.1872378187943826 	|  theta: tensor([[0.0501, 2.9985]], requires_grad=True)  |  Time needed: 0:00:09.503719  
Run: 6	| Iteration: 700 	| Log-Likelihood:-3.1872377254989575 	|  theta: tensor([[0.0501, 2.9988]], requires_grad=True)  |  Time needed: 0:00:09.582346  
Run: 6	| Iteration: 800 	| Log-Likelihood:-3.1872377343266196 	|  theta: tensor([[0.0501, 2.9988]], requires_grad=True)  |  Time needed: 0:00:09.408098  
Run: 6	| Iteration: 900 	| Log-Likelihood:-3.187237737584247 	|  theta: tensor([[0.0501, 2.9989]], requires_grad=True)  |  Time needed: 0:00:09.430640  
Run: 7	| Iteration: 0 	| Log-Likelihood:-5.564680138849629 	|  theta: tensor([

Run: 11	| Iteration: 800 	| Log-Likelihood:-3.187237734010984 	|  theta: tensor([[0.0501, 2.9989]], requires_grad=True)  |  Time needed: 0:00:09.545120  
Run: 11	| Iteration: 900 	| Log-Likelihood:-3.187237737584247 	|  theta: tensor([[0.0501, 2.9989]], requires_grad=True)  |  Time needed: 0:00:09.499638  
Run: 12	| Iteration: 0 	| Log-Likelihood:-3.3735121898983773 	|  theta: tensor([[0.0702, 2.4527]], requires_grad=True)  |  Time needed: 0:00:00.097315  
Run: 12	| Iteration: 100 	| Log-Likelihood:-3.203844015556984 	|  theta: tensor([[0.0502, 2.8374]], requires_grad=True)  |  Time needed: 0:00:09.472834  
Run: 12	| Iteration: 200 	| Log-Likelihood:-3.188601647195543 	|  theta: tensor([[0.0501, 2.9527]], requires_grad=True)  |  Time needed: 0:00:09.435150  
Run: 12	| Iteration: 300 	| Log-Likelihood:-3.1873488688651705 	|  theta: tensor([[0.0501, 2.9857]], requires_grad=True)  |  Time needed: 0:00:09.466254  
Run: 12	| Iteration: 400 	| Log-Likelihood:-3.187246814396041 	|  theta: ten

Run: 17	| Iteration: 200 	| Log-Likelihood:-3.1904627308266917 	|  theta: tensor([[0.4328, 1.0570]], requires_grad=True)  |  Time needed: 0:00:09.315235  
Run: 17	| Iteration: 300 	| Log-Likelihood:-3.1904627327541157 	|  theta: tensor([[0.4329, 1.0570]], requires_grad=True)  |  Time needed: 0:00:09.331843  
Run: 17	| Iteration: 400 	| Log-Likelihood:-3.190462702927815 	|  theta: tensor([[0.4329, 1.0570]], requires_grad=True)  |  Time needed: 0:00:09.502967  
Run: 17	| Iteration: 500 	| Log-Likelihood:-3.190462702927815 	|  theta: tensor([[0.4329, 1.0570]], requires_grad=True)  |  Time needed: 0:00:09.457052  
Run: 17	| Iteration: 600 	| Log-Likelihood:-3.190462702927815 	|  theta: tensor([[0.4329, 1.0570]], requires_grad=True)  |  Time needed: 0:00:09.458332  
Run: 17	| Iteration: 700 	| Log-Likelihood:-3.190462702927815 	|  theta: tensor([[0.4329, 1.0570]], requires_grad=True)  |  Time needed: 0:00:09.521498  
Run: 17	| Iteration: 800 	| Log-Likelihood:-3.190462702927815 	|  theta: t

Run: 22	| Iteration: 600 	| Log-Likelihood:-3.1904626731256314 	|  theta: tensor([[0.4329, 1.0570]], requires_grad=True)  |  Time needed: 0:00:09.586154  
Run: 22	| Iteration: 700 	| Log-Likelihood:-3.1904626731256314 	|  theta: tensor([[0.4329, 1.0570]], requires_grad=True)  |  Time needed: 0:00:09.603714  
Run: 22	| Iteration: 800 	| Log-Likelihood:-3.1904626731256314 	|  theta: tensor([[0.4329, 1.0570]], requires_grad=True)  |  Time needed: 0:00:09.553588  
Run: 22	| Iteration: 900 	| Log-Likelihood:-3.1904626731256314 	|  theta: tensor([[0.4329, 1.0570]], requires_grad=True)  |  Time needed: 0:00:09.537903  
Run: 23	| Iteration: 0 	| Log-Likelihood:-4.839580678395582 	|  theta: tensor([[0.2735, 4.5123]], requires_grad=True)  |  Time needed: 0:00:00.094138  
Run: 23	| Iteration: 100 	| Log-Likelihood:-3.305516181906513 	|  theta: tensor([[0.0501, 3.4285]], requires_grad=True)  |  Time needed: 0:00:09.414758  
Run: 23	| Iteration: 200 	| Log-Likelihood:-3.1967910479847417 	|  theta: 

...

Run: 310	| Iteration: 200 	| Log-Likelihood:-3.201402974902143 	|  theta: tensor([[0.0501, 3.1477]], requires_grad=True)  |  Time needed: 0:00:09.245606  
Run: 310	| Iteration: 300 	| Log-Likelihood:-3.188387036293282 	|  theta: tensor([[0.0501, 3.0413]], requires_grad=True)  |  Time needed: 0:00:09.198406  
Run: 310	| Iteration: 400 	| Log-Likelihood:-3.1873311770153556 	|  theta: tensor([[0.0501, 3.0110]], requires_grad=True)  |  Time needed: 0:00:09.087790  
Run: 310	| Iteration: 500 	| Log-Likelihood:-3.1872453169351256 	|  theta: tensor([[0.0501, 3.0023]], requires_grad=True)  |  Time needed: 0:00:09.189892  
Run: 310	| Iteration: 600 	| Log-Likelihood:-3.187238363303035 	|  theta: tensor([[0.0501, 2.9999]], requires_grad=True)  |  Time needed: 0:00:09.112867  
Run: 310	| Iteration: 700 	| Log-Likelihood:-3.1872377915553582 	|  theta: tensor([[0.0501, 2.9992]], requires_grad=True)  |  Time needed: 0:00:09.109608  
Run: 310	| Iteration: 800 	| Log-Likelihood:-3.1872377714395292 	| 

Run: 315	| Iteration: 500 	| Log-Likelihood:-3.190462702927815 	|  theta: tensor([[0.4329, 1.0570]], requires_grad=True)  |  Time needed: 0:00:09.141047  
Run: 315	| Iteration: 600 	| Log-Likelihood:-3.190462702927815 	|  theta: tensor([[0.4329, 1.0570]], requires_grad=True)  |  Time needed: 0:00:09.076517  
Run: 315	| Iteration: 700 	| Log-Likelihood:-3.190462702927815 	|  theta: tensor([[0.4329, 1.0570]], requires_grad=True)  |  Time needed: 0:00:09.219529  
Run: 315	| Iteration: 800 	| Log-Likelihood:-3.190462702927815 	|  theta: tensor([[0.4329, 1.0570]], requires_grad=True)  |  Time needed: 0:00:09.075453  
Run: 315	| Iteration: 900 	| Log-Likelihood:-3.190462702927815 	|  theta: tensor([[0.4329, 1.0570]], requires_grad=True)  |  Time needed: 0:00:09.064403  
Run: 316	| Iteration: 0 	| Log-Likelihood:-4.058940481159972 	|  theta: tensor([[0.2206, 4.0786]], requires_grad=True)  |  Time needed: 0:00:00.091095  
Run: 316	| Iteration: 100 	| Log-Likelihood:-3.2474593348194802 	|  thet

Run: 320	| Iteration: 800 	| Log-Likelihood:-3.1872377657243565 	|  theta: tensor([[0.0501, 2.9989]], requires_grad=True)  |  Time needed: 0:00:09.120125  
Run: 320	| Iteration: 900 	| Log-Likelihood:-3.1872377563521925 	|  theta: tensor([[0.0501, 2.9989]], requires_grad=True)  |  Time needed: 0:00:09.182979  
Run: 321	| Iteration: 0 	| Log-Likelihood:-3.3373272789269093 	|  theta: tensor([[0.2183, 3.2103]], requires_grad=True)  |  Time needed: 0:00:00.090272  
Run: 321	| Iteration: 100 	| Log-Likelihood:-3.1895556880680527 	|  theta: tensor([[0.0501, 3.0591]], requires_grad=True)  |  Time needed: 0:00:09.244596  
Run: 321	| Iteration: 200 	| Log-Likelihood:-3.187426163960753 	|  theta: tensor([[0.0501, 3.0160]], requires_grad=True)  |  Time needed: 0:00:09.275969  
Run: 321	| Iteration: 300 	| Log-Likelihood:-3.1872530607111065 	|  theta: tensor([[0.0501, 3.0038]], requires_grad=True)  |  Time needed: 0:00:09.292047  
Run: 321	| Iteration: 400 	| Log-Likelihood:-3.187239010583035 	|  

Run: 326	| Iteration: 100 	| Log-Likelihood:-3.269254812204426 	|  theta: tensor([[0.0501, 3.3567]], requires_grad=True)  |  Time needed: 0:00:09.287728  
Run: 326	| Iteration: 200 	| Log-Likelihood:-3.1938675946095576 	|  theta: tensor([[0.0501, 3.1007]], requires_grad=True)  |  Time needed: 0:00:09.419726  
Run: 326	| Iteration: 300 	| Log-Likelihood:-3.187776237830788 	|  theta: tensor([[0.0501, 3.0279]], requires_grad=True)  |  Time needed: 0:00:09.357100  
Run: 326	| Iteration: 400 	| Log-Likelihood:-3.1872815198717155 	|  theta: tensor([[0.0501, 3.0072]], requires_grad=True)  |  Time needed: 0:00:09.296988  
Run: 326	| Iteration: 500 	| Log-Likelihood:-3.187241318068376 	|  theta: tensor([[0.0501, 3.0012]], requires_grad=True)  |  Time needed: 0:00:09.288499  
Run: 326	| Iteration: 600 	| Log-Likelihood:-3.1872380382124006 	|  theta: tensor([[0.0501, 2.9995]], requires_grad=True)  |  Time needed: 0:00:09.273437  
Run: 326	| Iteration: 700 	| Log-Likelihood:-3.1872377498590043 	| 

KeyboardInterrupt: 

In [36]:
total = glob_max_counter+ loc_max_counter+ rest_counter
print(f'Estimated q_1: {glob_max_counter/total}\n')
print(f'Estimated q_2: {loc_max_counter/total}\n')

Estimated q_1: 0.3597560975609756

Estimated q_2: 0.6402439024390244



print(glob_max_counter, loc_max_counter, rest_counter)

In [ ]:
'''
Gradient Ascent
'''
#X = torch.from_numpy(X)
# Set parameters here
n_iterations = 5000 #max number of iterations # TODO: alternatively/ additionally a error-based threshold?
lr = 0.001 # learning rate / stepsize
n_runs = 20

# Initializing Loss as minus infinity to make sure first run achieves higher likelihood
max_likelihood = -1*np.inf
# trajectory_dict is a cache to save the gradient ascent trajectory of all gradient ascent runs
trajectory_dict = {}

# Running Gradient Ascent multiple (M=n_runs) times
for run in range(n_runs):

    # Create/ Initialize variable ' TODO: make initialization more flexible
    theta = torch.tensor([[uniform.Uniform(0., .6).sample(),uniform.Uniform(0., 5.).sample()]], requires_grad = True)

    # Run complete Gradient ascent
    theta, L, trajectory = gradient_ascent_torch(func = LogLikelihood,
                                           param=theta,
                                           data=X,
                                           max_iterations=n_iterations,
                                           learningrate=lr,
                                           run_id=run,
                                           print_info=True)

    # Save optimization trajectory
    trajectory_dict.update({run : trajectory})
    # Updating Quantities if new max is found

    # compare likelihood value to previous runs
    if L > max_likelihood:
        # This takes forever if n is large. As it is torch implementation I don't see a way to get this faster
        print(f'New Maximum found! old:{max_likelihood} -> new:{L}')

        # Update highest likelihood and theta estimate
        max_likelihood = L
        theta_hat = theta.clone().data.numpy()

        # get derivatives
        Scores, Hessian = get_derivatives_torch(func=LogLikelihood,
                                                param=theta,
                                                data=X,
                                                print_dims=True)


CI = normal_CI(0.05, Scores, Hessian, theta_hat)

print(f'theta:\n {theta_hat}')
print(f'normal CI borders:\n {CI}')


# Starting Plot
if make_plots:
    fig, ax = plt.subplots(1,1, figsize = (7,7))
    for run in range(n_runs):
        trajectory = trajectory_dict[run]
        m = len(trajectory_dict[run])
        ax.plot([trajectory[i][0,1] for i in range(m)], [trajectory[i][0,0] for i in range(m)], label = f'run: {run}')
    ax.imshow(Colourplot, cmap='Reds', extent = [0,5,0,.6], aspect='auto') #extent = [0,5,0,6]
    ax.set_xlabel('mu')
    ax.set_ylabel('rho')
    plt.title('Contourplot of Log-Likelihood function with gradient ascent trajectories')
    plt.show()

In [46]:
def theta_n_M(data, n_runs, func, max_iterations = 1000, learningrate = 0.01, print_info=True):
    
    # Initializing Loss as minus infinity to make sure first run achieves higher likelihood
    max_likelihood = -1*np.inf
    # trajectory_dict is a cache to save the gradient ascent trajectory of all gradient ascent runs
    trajectory_dict = {}

    # Running Gradient Ascent multiple (M=n_runs) times
    for run in range(n_runs):

        # Create/ Initialize variable ' TODO: make initialization more flexible
        theta = torch.tensor([[uniform.Uniform(0., .6).sample(),uniform.Uniform(0., 5.).sample()]], requires_grad = True)

        # Run complete Gradient ascent
        theta, L, trajectory = gradient_ascent_torch(func = func,
                                               param=theta,
                                               data=data,
                                               max_iterations=max_iterations,
                                               learningrate=learningrate,
                                               run_id=run,
                                               print_info=print_info)

        # Save optimization trajectory
        trajectory_dict.update({run : trajectory})
        # Updating Quantities if new max is found

        # compare likelihood value to previous runs
        if L > max_likelihood:
            # This takes forever if n is large. As it is torch implementation I don't see a way to get this faster
            print(f'New Maximum found! old:{max_likelihood} -> new:{L}')

            # Update highest likelihood and theta estimate
            max_likelihood = L
            theta_hat = theta.clone().data.numpy()

    # get derivatives
    Scores, Hessian = get_derivatives_torch(func=func,
                                            param=torch.tensor(theta_hat, requires_grad = True),
                                            data=data,
                                            print_dims=True)


            
    return theta_hat, Scores, Hessian, trajectory_dict

In [47]:
'''
Gradient Ascent
'''
X2 = gaussian_mixture_model_sample(10**6, means, covs, weights, test=False) # we can alternatively use pytorchs MixtureFamily
X2 = torch.from_numpy(X2)

theta_hat_test, Scores_test, Hessian_test, trajectory_dict_test = theta_n_M(data = 2,
                                                                            n_runs = 5,
                                                                            func = LogLikelihood,
                                                                            max_iterations = 200,
                                                                            learningrate = 0.01,
                                                                            print_info=True)


Run: 1	| Iteration: 0 	| Log-Likelihood:-49.4559211730957 	|  theta: tensor([[0.1246, 4.2537]], requires_grad=True)  |  Time needed: 0:00:00.001253  
Run: 1	| Iteration: 100 	| Log-Likelihood:-48.80320739746094 	|  theta: tensor([[-0.6651,  4.2537]], requires_grad=True)  |  Time needed: 0:00:00.070844  
New Maximum found! old:-inf -> new:-48.53196334838867
Run: 2	| Iteration: 0 	| Log-Likelihood:-0.2195502519607544 	|  theta: tensor([[0.4274, 2.0008]], requires_grad=True)  |  Time needed: 0:00:00.001081  
Run: 2	| Iteration: 100 	| Log-Likelihood:1.0790475606918335 	|  theta: tensor([[1.4816, 2.0000]], requires_grad=True)  |  Time needed: 0:00:00.072013  
New Maximum found! old:-48.53196334838867 -> new:1.4030635356903076
Run: 3	| Iteration: 0 	| Log-Likelihood:-39.919315338134766 	|  theta: tensor([[0.5263, 3.3405]], requires_grad=True)  |  Time needed: 0:00:00.001076  
Run: 3	| Iteration: 100 	| Log-Likelihood:1.099851131439209 	|  theta: tensor([[1.5125, 2.0000]], requires_grad=True